In [1]:
from path import Path
def read_txt(file_path: Path) -> str:
    with open(file_path) as file:
        return file.read() 

In [2]:
def form_list_of_T_entity(text: str) -> list[list[str]]:
    splited_text = text.split('\n')
    filtered = []
    for line in splited_text:
        if len(line) > 1 and line[0] == 'T' and not ';' in line:  
            data = line.split('\t')[1].split(' ')
            data[1] = int(data[1])
            data[2] = int(data[2])
            filtered.append(data)
    return filtered


In [3]:
def filter_list_of_T_entity(data: list[list[str]]) -> list[list[str]]:
    return sorted(data, key=lambda entity: (entity[1], - entity[2]))

In [4]:
def filter_spans(data:  list[list[str]]) ->  list[list[str]]:
    span_arange = -1
    result = []
    for entity in data:
        if entity[1] > span_arange:
            result.append(entity)
            span_arange = entity[2]
    return result
def form_lables_list(file_path: Path):
    text = read_txt(file_path)
    lables = form_list_of_T_entity(text)
    lables = filter_list_of_T_entity(lables)
    return filter_spans(lables)

In [5]:
form_lables_list('test/1130.ann')


[['NATIONALITY', 0, 9],
 ['PROFESSION', 10, 16],
 ['PERSON', 17, 28],
 ['EVENT', 29, 38],
 ['ORGANIZATION', 40, 46],
 ['CITY', 49, 55],
 ['NATIONALITY', 58, 67],
 ['PROFESSION', 68, 74],
 ['PERSON', 75, 86],
 ['PROFESSION', 98, 141],
 ['CITY', 143, 149],
 ['AGE', 163, 172],
 ['PROFESSION', 173, 181],
 ['PROFESSION', 278, 304],
 ['PERSON', 305, 311],
 ['NATIONALITY', 320, 331],
 ['PROFESSION', 332, 343],
 ['PERSON', 344, 361],
 ['DATE', 378, 396],
 ['PERSON', 397, 403],
 ['ORGANIZATION', 416, 420],
 ['AWARD', 440, 477]]

In [6]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

2022-11-29 16:10:51.222637: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 16:10:51.467376: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-29 16:10:52.590774: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-29 16:10:52.591078: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [7]:
def form_bio_mapping(mapping, text, labels):
    tokens = []
    bio_label = []
    for token in mapping:
        tokens.append(text[token[0]:token[1]])
        for lable in labels:
            if lable[1] == token[0]:
#                 print('BBBBB')
                bio_label.append('B-'+lable[0])
                break
            elif lable[1] < token[0] and lable[2] >= token[1]:
#                 print('IIII')
                bio_label.append('I-'+lable[0])
                break
            elif lable[1] > token[0]:
#                 print("OOO")
                bio_label.append('O')
                break
        else:
            bio_label.append('O')
    return tokens, mapping, bio_label
    
def form_span_from_text(file_path: Path):
    text = read_txt(file_path)
    offset_mapping = tokenizer(text,return_offsets_mapping = True, add_special_tokens=False).offset_mapping
    ann_file = file_path.split('.')[0] + '.ann'
    labels = form_lables_list(ann_file)
    tokens, mapping, bio_label = form_bio_mapping(offset_mapping, text, labels)
    return tokens, mapping, bio_label

    

In [8]:
tokens, mapping, bio_label = form_span_from_text('train/90693_text.txt')
print(len(tokens), len(mapping), len(bio_label))
bio_label

318 318 318


['B-NATIONALITY',
 'I-NATIONALITY',
 'B-PROFESSION',
 'O',
 'B-PENALTY',
 'I-PENALTY',
 'I-PENALTY',
 'O',
 'O',
 'O',
 'B-PERSON',
 'I-PERSON',
 'B-PERSON',
 'I-PERSON',
 'B-NATIONALITY',
 'I-NATIONALITY',
 'B-PROFESSION',
 'O',
 'O',
 'O',
 'O',
 'B-COUNTRY',
 'O',
 'O',
 'O',
 'O',
 'B-DATE',
 'I-DATE',
 'I-DATE',
 'I-DATE',
 'I-DATE',
 'I-DATE',
 'I-DATE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'I-ORGANIZATION',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PENALTY',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PROFESSION',
 'I-PROFESSION',
 'I-PROFESSION',
 'O',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'B-PERSON',
 'I-PERSON',
 'O',
 'B-PERSON',
 'I-PERSON',
 'I-PERSON',
 'I-PERSON',
 'I-PERSON',
 'I-PERSON',
 'I-PERSON',
 'O',
 'B-AGE',
 'I-AGE',
 'I-AGE',
 'B-PROFESSION',
 'O',
 'B-CITY',
 'I-CITY',
 'O',
 'O',
 'O',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'I-ORGANIZATION',
 'O',
 'B-CITY',
 

In [9]:
import os
def form_data_sets(library: Path):
    names = os.listdir(os.getcwd()+'/'+library)
    token_seq = []
    spans = []
    lables = []
    for name in names:
        if name.split('.')[1] == 'txt':
            tmp_token_seq, tmp_spans, tmp_lables = form_span_from_text(os.getcwd()+'/'+library+'/'+name)
            token_seq.append(tmp_token_seq)
            spans.append(tmp_spans)
            lables.append(tmp_lables)

            tmp_token_seq = tmp_spans = tmp_lables = []
    return token_seq, spans, lables

In [10]:
train_token_seq, train_spans_seq, train_label_seq = form_data_sets('train')
valid_token_seq, valid_spans_seq, valid_label_seq = form_data_sets('dev')
test_token_seq, test_spans_seq, test_label_seq = form_data_sets('test')
print(train_label_seq)

Token indices sequence length is longer than the specified maximum sequence length for this model (2392 > 2048). Running this sequence through the model will result in indexing errors


[['B-PROFESSION', 'I-PROFESSION', 'O', 'O', 'O', 'O', 'B-DISEASE', 'I-DISEASE', 'B-PERSON', 'I-PERSON', 'I-PERSON', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'B-PROFESSION', 'I-PROFESSION', 'B-PERSON', 'I-PERSON', 'I-PERSON', 'I-PERSON', 'B-PERSON', 'I-PERSON', 'I-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DISEASE', 'I-DISEASE', 'O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'B-DISEASE', 'I-DISEASE', 'I-DISEASE', 'I-DISEASE', 'I-DISEASE', 'I-DISEASE', 'O', 'O', 'O', 'B-DATE', 'I-DATE', 'O', 'O', 'B-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [11]:
from collections import Counter, defaultdict, namedtuple
token2cnt = Counter([token for sentence in train_token_seq for token in sentence])

In [12]:
token2cnt.most_common(10)

[(',', 11454),
 ('.', 10040),
 ('в', 7373),
 ('и', 3851),
 ('-', 3010),
 ('на', 2660),
 ('»', 2152),
 ('«', 2151),
 ('с', 1937),
 ('что', 1532)]

In [13]:
from typing import Tuple, List, Dict, Any
def get_token2idx(
    token2cnt: Dict[str, int],
    min_count: int,
) -> Dict[str, int]:
    """
    Get mapping from tokens to indices to use with Embedding layer.
    """

    token2idx: Dict[str, int] = {}

    # YOUR CODE HERE
    token2idx = {'<PAD>': 0, '<UNK>': 1}
    
    token_len = len(token2idx)
    for token, count in token2cnt.items():
        if count >= min_count:
            token2idx[token] = token_len
            token_len += 1

    return token2idx



In [14]:
token2idx = get_token2idx(token2cnt, min_count=2)



In [15]:
def sort_labels_func(x: str) -> int:
    if x == "O":
        return 0
    elif x.startswith("B-"):
        return 1
    else:
        return 2

label_set = sorted(
    set(label for sentence in train_label_seq for label in sentence),
    key=lambda x: (sort_labels_func(x), x),
)


In [16]:
label_set

['O',
 'B-AGE',
 'B-AWARD',
 'B-CITY',
 'B-COUNTRY',
 'B-CRIME',
 'B-DATE',
 'B-DISEASE',
 'B-DISTRICT',
 'B-EVENT',
 'B-FACILITY',
 'B-FAMILY',
 'B-IDEOLOGY',
 'B-LANGUAGE',
 'B-LAW',
 'B-LOCATION',
 'B-MONEY',
 'B-NATIONALITY',
 'B-NUMBER',
 'B-ORDINAL',
 'B-ORGANIZATION',
 'B-PENALTY',
 'B-PERCENT',
 'B-PERSON',
 'B-PRODUCT',
 'B-PROFESSION',
 'B-RELIGION',
 'B-STATE_OR_PROVINCE',
 'B-TIME',
 'B-WORK_OF_ART',
 'I-AGE',
 'I-AWARD',
 'I-CITY',
 'I-COUNTRY',
 'I-CRIME',
 'I-DATE',
 'I-DISEASE',
 'I-DISTRICT',
 'I-EVENT',
 'I-FACILITY',
 'I-FAMILY',
 'I-IDEOLOGY',
 'I-LANGUAGE',
 'I-LAW',
 'I-LOCATION',
 'I-MONEY',
 'I-NATIONALITY',
 'I-NUMBER',
 'I-ORDINAL',
 'I-ORGANIZATION',
 'I-PENALTY',
 'I-PERCENT',
 'I-PERSON',
 'I-PRODUCT',
 'I-PROFESSION',
 'I-RELIGION',
 'I-STATE_OR_PROVINCE',
 'I-TIME',
 'I-WORK_OF_ART']

In [17]:
def get_label2idx(label_set: List[str]) -> Dict[str, int]:
    """
    Get mapping from labels to indices.
    """

    label2idx: Dict[str, int] = {}

    # YOUR CODE HERE
    label_len = len(label2idx)
    for label in label_set:
        label2idx[label] = label_len
        label_len += 1
    return label2idx

In [18]:
label2idx = get_label2idx(label_set)

In [19]:
for token, idx in list(token2idx.items())[:10]:
    print(f"{token}\t{idx}")

<PAD>	0
<UNK>	1
Президент	2
Узбекистана	3
находится	4
в	5
реанимации	6
с	7
инсульт	8
ом	9


In [20]:
for label, idx in label2idx.items():
    print(f"{label}\t{idx}")

O	0
B-AGE	1
B-AWARD	2
B-CITY	3
B-COUNTRY	4
B-CRIME	5
B-DATE	6
B-DISEASE	7
B-DISTRICT	8
B-EVENT	9
B-FACILITY	10
B-FAMILY	11
B-IDEOLOGY	12
B-LANGUAGE	13
B-LAW	14
B-LOCATION	15
B-MONEY	16
B-NATIONALITY	17
B-NUMBER	18
B-ORDINAL	19
B-ORGANIZATION	20
B-PENALTY	21
B-PERCENT	22
B-PERSON	23
B-PRODUCT	24
B-PROFESSION	25
B-RELIGION	26
B-STATE_OR_PROVINCE	27
B-TIME	28
B-WORK_OF_ART	29
I-AGE	30
I-AWARD	31
I-CITY	32
I-COUNTRY	33
I-CRIME	34
I-DATE	35
I-DISEASE	36
I-DISTRICT	37
I-EVENT	38
I-FACILITY	39
I-FAMILY	40
I-IDEOLOGY	41
I-LANGUAGE	42
I-LAW	43
I-LOCATION	44
I-MONEY	45
I-NATIONALITY	46
I-NUMBER	47
I-ORDINAL	48
I-ORGANIZATION	49
I-PENALTY	50
I-PERCENT	51
I-PERSON	52
I-PRODUCT	53
I-PROFESSION	54
I-RELIGION	55
I-STATE_OR_PROVINCE	56
I-TIME	57
I-WORK_OF_ART	58


In [21]:
import torch
class NERDataset(torch.utils.data.Dataset):
    """
    PyTorch Dataset for NER.
    """

    def __init__(
        self,
        token_seq: List[List[str]],
        label_seq: List[List[str]],
        token2idx: Dict[str, int],
        label2idx: Dict[str, int],
    ):
        self.token2idx = token2idx
        self.label2idx = label2idx

        self.token_seq = [self.process_tokens(tokens, token2idx) for tokens in token_seq]
        self.label_seq = [self.process_labels(labels, label2idx) for labels in label_seq]

    def __len__(self):
        return len(self.token_seq)

    def __getitem__(
        self,
        idx: int,
    ) -> Tuple[torch.LongTensor, torch.LongTensor]:
        # YOUR CODE HERE
        return torch.LongTensor(self.token_seq[idx]), torch.LongTensor(self.label_seq[idx])

    @staticmethod
    def process_tokens(
        tokens: List[str],
        token2idx: Dict[str, int],
        unk: str = "<UNK>",
    ) -> List[int]:
        """
        Transform list of tokens into list of tokens' indices.
        """
        # YOUR CODE HERE
        result = []
        for token in tokens:
            result += [token2idx.get(token, token2idx.get(unk))]
        return result

    @staticmethod
    def process_labels(
        labels: List[str],
        label2idx: Dict[str, int],
    ) -> List[int]:
        """
        Transform list of labels into list of labels' indices.
        """
        # YOUR CODE HERE
        result = []
        for label in labels:
            result += [label2idx.get(label)]
        return result

In [22]:
train_dataset = NERDataset(
    token_seq=train_token_seq,
    label_seq=train_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)
valid_dataset = NERDataset(
    token_seq=valid_token_seq,
    label_seq=valid_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)
test_dataset = NERDataset(
    token_seq=test_token_seq,
    label_seq=test_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)

In [23]:
train_dataset[0]

(tensor([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,
          16,  17,  18,   3,  10,  19,  11,  20,  21,  22,  11,  20,  23,  24,
          25,  26,  27,  28,   5,  29,   7,   8,   9,  30,  21,  31,  32,   5,
           1,   1,  33,  34,   1,   1,  35,   1,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  24,  48,  49,   5,  50,  51,   7,  52,
          53,   5,   1,  54,  30,  55,  56,  57,  58,  59,  60,  61,  62,  24,
          63,  64,   5,  65,  66,  67,  68,  69,   4,   5,  70,  71,  72,  30,
          73,  74,  75,  76,  77,  30,  78,  79,  80,  81,  82,  83,  84,  85,
          86,  87,  74,  88,  89,  30,  90,  91,  92,  93,  94,   7,  95,  96,
          97,  98,   1,  99,  69, 100, 101, 102,  37, 103, 104, 105, 106,  69,
         103, 107, 108, 109, 110,  30,  90, 111, 112, 113,   1, 114,  24, 115,
         116, 117, 118, 119,  69, 120, 121, 122, 123,   1, 124,  30, 125,  24,
          25, 126, 127, 128,  18, 129,  23,  44, 130

In [24]:
valid_dataset[0]

(tensor([ 7414,  1185, 14859,   931,  1984,  3732,     1,    40, 11187,    44,
           403, 12183,    60,   829,  1819,  1820, 14859,   931,  1984,  3732,
             1,    40, 11187,    44,    30,  2789,   134,   595,  1083,  1020,
         17137,  2673, 13000,    37,  2656,  3893,    30,   265,   948,   947,
            24,    25,  3818,     5,  6573,   127,  6761, 14859,   931,  1984,
          3732,     5,  2792,   437, 12957,  8724, 12345,    30,     1,    40,
         11187,    44,    24,  1733,   776, 11657,   127,   202, 11187,    44,
             5,  8298,   968,  8611,   159, 11588,    24,  7669,   718,   136,
         18156,    30,  1632,  8695,     1,  8095,     1,     5,   266,  2137,
            16,    24,   207,  1417, 14786,    37,  8039,  1432,   245,     1,
            30,     1,  7637, 14859,   931,  1984,  3732,     5,   948,    44,
          2782,  2114, 13000,   428,   764,    44,  2821,    24,    25, 11187,
            44,  9485,  1295,   395,  1207,    30,  

In [25]:
test_dataset[0]

(tensor([ 9949,  2751,   178,   414,     1,   199,  6066,     7,  1899,  5630,
          8080,  6083,  1824,    93,  8486,  2411,     1, 15781,  1203,  9949,
           826, 11316,   414,  1733,    16,  8919,     1,  6083,   433,  1824,
            93,  8486,  2411,    60, 13278,    69,  2385,  1576, 10662,   169,
            10,  2149,   129,   172, 14146,  3301,   457,  9556,   620, 15781,
          1203,    60, 17398,   199,   647,  6066,     7,  7066, 10616,  8098,
             5,  2352,  5605,     1,  2398,    30,  2588,  5500,  6077, 15416,
             1,   197,  9989,  2149,  1325,     1,   177,     1,     7,   127,
          8942, 14560,     5,  1082,    24,   884,  1306,  1733,    16,    24,
          1824,    93,  8486,  2411,    69, 15781,  1203,   169,     1, 12863,
          3914,     5,     1,  2514,    93,  5223,  2462,     1,    69,  6066,
             7,  7131,   172,    30,     1,    69,     1,     1,  1824,    93,
          8486,  2411,  1850,    24, 10955, 11373, 1

In [26]:
class NERCollator:
    """
    Collator that handles variable-size sentences.
    """

    def __init__(
        self,
        token_padding_value: int,
        label_padding_value: int,
    ):
        self.token_padding_value = token_padding_value
        self.label_padding_value = label_padding_value

    def __call__(
        self,
        batch: List[Tuple[torch.LongTensor, torch.LongTensor]],
    ) -> Tuple[torch.LongTensor, torch.LongTensor]:

        tokens, labels = zip(*batch)
        tokens = torch.nn.utils.rnn.pad_sequence(tokens,
                                               batch_first=True,
                                               padding_value=self.token_padding_value)
        labels = torch.nn.utils.rnn.pad_sequence(labels,
                                                batch_first=True,
                                                padding_value=self.label_padding_value)
        return {
        "input_ids": tokens,
        "labels": labels,
        }

In [27]:
collator = NERCollator(
    token_padding_value=tokenizer.pad_token_id,
    label_padding_value=-100
)

In [28]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [29]:
model = AutoModelForTokenClassification.from_pretrained("cointegrated/rubert-tiny2", num_labels=len(label_set))

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized 

In [30]:
training_args = TrainingArguments(
    output_dir=os.getcwd()+'/output_model',
    learning_rate=8e-5,
    weight_decay=1e-3,
    lr_scheduler_type='cosine',
    full_determinism=False,
    seed=55,
    per_device_train_batch_size=2,
    num_train_epochs=50,
    evaluation_strategy='steps',
    eval_steps=500)

In [31]:
trainer = Trainer(
model=model,
args=training_args,
data_collator=collator,
train_dataset=train_dataset,
eval_dataset=valid_dataset
)

In [32]:
trainer.train()

/home/dmitry/PycharmProjects/text/venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 746
  Num Epochs = 50
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 18650


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 152.00 MiB (GPU 0; 1.96 GiB total capacity; 1.16 GiB already allocated; 64.94 MiB free; 1.27 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF